# Calcul du Internal Product
Le but de ce carnet jupyter est de faire le calcul du produit scalaire
$$
\braket{x|\psi}
$$

avec
$$
\ket{\psi}=P_{\text{G}}P_{\text{J}}\ket{\phi_\text{Pfaff}},
$$
afin de pouvoir calculer la probabilité d'une update. Ce carnet va aussi contenir des exemples d'utilisation des _fast-updates_.

Pour rouler le code présent dans ce carnet, il faut avoir compilé la librairie avec la _feature_ `python-interface`, puis avoir exposé l'interface à l'environnement python à l'aide de `maturin`. Le script `build.sh` présent dans
ce répertoire permet la création d'un environnement virtuel et l'installation de la librairie `impurity`.

In [1]:
import impurity as imp
NSITES = 8
spin_up = 101
spin_down = 164
new_spin_up = 165

## Mise en situation
Pour l'entièreté de ce carnet, le problème qui sera considéré sera une chaine de $8$ spins ayant une connectivité périodique. L'état qui sera considéré sera une configuration $s_{\uparrow}=[01100101]=101$ pour les spins _up_ ainsi qu'une configuration $s_{\downarrow}=[10100100]=164$ pour les spins _down_.

Ajouter une figure

Pour les fast-updates, le _hopping_ sera de spin up et passera de l'indice $1$ à l'indice $0$. La nouvelle configuration _up_ sera donc $s_{\uparrow}=[10100101]=165$.

Ajouter une figure

## Calcul du Projecteur Gutzwiller $P_{\text{G}}$
On utilise la définition du projecteur de Gutzwiller

$$
P_{\text{G}}=e^{\sum_i g_in_{i\uparrow}n_{i\downarrow}},
$$
ou sous sa forme plus commode numériquement
$$
\ln P_{\text{G}}=\sum_i g_i n_{i\uparrow}n_{i\downarrow}.
$$
La somme sur $i$ représente la somme sur l'entièreté des sites. Les $g_i$ sont les paramètres variationnels du projecteur. Nous fixons ces paramètres aux valeurs:

$$
\mathbf{g}=\begin{pmatrix}
-0.23 &-0.59 & -0.58 & -0.8 & -0.57 & -0.47 & 0.4 & -0.62
\end{pmatrix}
$$

### Calcul explicite
Les seuls termes de la somme qui contribuent sont ceux pour lesquels le site contient un spin _up_ et un spin _down_, donc équivalent au produit matriciel suivant. Les indices à l'intérieur du vecteur colonne sont donnés par l'opération
_et bitwise_ entre $s_\uparrow$ et $s_\downarrow$, $\zeta=s_\uparrow\&s_\downarrow$.

$$
\begin{align}
\sum_i g_i n_{i\uparrow}n_{i\downarrow}&=\begin{pmatrix}
-0.23 &-0.59 & -0.58 & -0.8 & -0.57 & -0.47 & 0.4 & -0.62
\end{pmatrix}
\begin{pmatrix}
0 \\ 0 \\ 1 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0
\end{pmatrix}\\
&=-0.58 - 0.47=-1.05
\end{align}
$$

Pour après le _hopping_, les termes qui contribuent change, soit

$$
\begin{align}
\sum_i g_i n_{i\uparrow}n_{i\downarrow}&=\begin{pmatrix}
-0.23 &-0.59 & -0.58 & -0.8 & -0.57 & -0.47 & 0.4 & -0.62
\end{pmatrix}
\begin{pmatrix}
1 \\ 0 \\ 1 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0
\end{pmatrix}\\
&=-0.23-0.58 - 0.47=-1.28
\end{align}
$$

### Calcul numérique

In [2]:
# Calcul pour la première configuration
gutzwiller_parameters = [-0.23, -0.59, -0.58, -0.8, -0.57, -0.47, 0.4, -0.62]
gutzwiller_coefficient = imp.gutzwiller_exponent(spin_up, spin_down, gutzwiller_parameters, NSITES)
print(gutzwiller_coefficient)

# Calcul après le hopping
gutzwiller_coefficient = imp.gutzwiller_fastupdate(
    gutzwiller_coefficient, gutzwiller_parameters, spin_up, spin_down, 1, 0, True
)
print(gutzwiller_coefficient)

-1.0499999999999998
-1.2799999999999998


![Gutzwiller](../target/criterion/Calcul%20du%20projecteur%20Gutzwiller/Calcul%20complet/report/lines.svg)
![Gutzwiller](../target/criterion/Calcul%20du%20projecteur%20Gutzwiller/Fast%20update/report/lines.svg)

## Calcul du Projecteur Jastrow $P_{\text{J}}$
On utilise la définition du projecteur de Jastrow

$$
\begin{align}
P_{\text{J}}&=e^{\frac12 \sum_{i\neq j} v_{ij}(n_i-1)(n_j -1)}\\
\ln P_{\text{J}}&=\frac12 \sum_{i\neq j} v_{ij}(n_i-1)(n_j -1)
\end{align}
$$
où les $v_{ij}$ sont les paramètres variationnels, tandis que les opérateurs $n_i$ sont les opérateurs nombre d'occupation sans égars au spin. La matrice $\mathbf{v}$ doit être symétrique, il s'agit d'un argument physique. Dans le code, aucune garantie n'est faite sur l'adressage de la matrice $\mathbf{v}$, mais cela pourrait être une optimisation future, ayant pour but d'augmenter le nombre de valeurs participant à la somme dans le cache. La matrice $\mathbf{v}$ est adressée plus souvent pour les éléments du triangle supérieur, mais ce n'est pas garanti.

$$
\mathbf{v}=\begin{pmatrix}
0 & -0.7 & 0.28 & -0.42 & 0.31 & 0.63 & -0.33 & 1\\
-0.7 & 0 & -0.11 & 0.38 & 0.06 & 0.44 & 0.93 & -0.32\\
0.28 & -0.11 & 0 & -0.07 & 0.26 & 0.99 & 0 & -0.94\\
-0.42 & 0.38 & -0.07 & 0 & -0.13 & 0.33 & -0.98 & 0.96\\
0.31 & 0.06 & 0.26 & -0.13 & 0 & -0.92 & 0.65 & -0.58\\
0.63 & 0.44 & 0.99 & 0.33 & -0.92 & 0 & 0.01 & -0.3 \\
-0.33 & 0.93 & 0 & -0.98 & 0.65 & 0.01 & 0 & 0.08\\
1 & -0.32 & -0.94 & 0.96 & -0.58 & -0.3 & 0.08 & 0
\end{pmatrix}
$$

La somme s'écrit comme le produit matriciel suivant, en utilisant le vecteur $\zeta=n_\uparrow+n_\downarrow-\mathbf{1}$,

$$
\begin{align}
n_{\uparrow}=[01100101]\\
n_{\downarrow}=[10100100]\\
\zeta=[0,0,1,-1,-1,1,-1,0]\\
\frac12 \sum_{i\neq j} v_{ij}(n_i-1)(n_j -1)=\\
\frac12\begin{pmatrix}
0 & 0 & 1 & -1 & -1 & 1 & -1 & 0
\end{pmatrix}
\begin{pmatrix}
0 & -0.7 & 0.28 & -0.42 & 0.31 & 0.63 & -0.33 & 1\\
-0.7 & 0 & -0.11 & 0.38 & 0.06 & 0.44 & 0.93 & -0.32\\
0.28 & -0.11 & 0 & -0.07 & 0.26 & 0.99 & 0 & -0.94\\
-0.42 & 0.38 & -0.07 & 0 & -0.13 & 0.33 & -0.98 & 0.96\\
0.31 & 0.06 & 0.26 & -0.13 & 0 & -0.92 & 0.65 & -0.58\\
0.63 & 0.44 & 0.99 & 0.33 & -0.92 & 0 & 0.01 & -0.3 \\
-0.33 & 0.93 & 0 & -0.98 & 0.65 & 0.01 & 0 & 0.08\\
1 & -0.32 & -0.94 & 0.96 & -0.58 & -0.3 & 0.08 & 0
\end{pmatrix}
\begin{pmatrix}
0 \\ 0 \\ 1 \\ -1 \\ -1 \\ 1 \\ -1 \\ 0
\end{pmatrix}\\
=\frac12
\begin{pmatrix}
0 & 0 & 1 & -1 & -1 & 1 & -1 & 0
\end{pmatrix}
\begin{pmatrix}\frac{27}{20}\\ 
-\frac{26}{25}\\ 
\frac{4}{5}\\ 
\frac{137}{100}\\ 
-\frac{59}{50}\\ 
\frac{157}{100}\\ 
\frac{17}{50}\\ 
-\frac{17}{10} \end{pmatrix}\\
=0.92
\end{align}
$$

On peut faire la même chose pour après le hopping, avec $\zeta=[1,-1,1,-1,-1,1,-1,0]$.
$$
\begin{align}
\frac12 \sum_{i\neq j} v_{ij}(n_i-1)(n_j -1)=\\
\frac12\begin{pmatrix}
1 & -1 & 1 & -1 & -1 & 1 & -1 & 0
\end{pmatrix}
\begin{pmatrix}
0 & -0.7 & 0.28 & -0.42 & 0.31 & 0.63 & -0.33 & 1\\
-0.7 & 0 & -0.11 & 0.38 & 0.06 & 0.44 & 0.93 & -0.32\\
0.28 & -0.11 & 0 & -0.07 & 0.26 & 0.99 & 0 & -0.94\\
-0.42 & 0.38 & -0.07 & 0 & -0.13 & 0.33 & -0.98 & 0.96\\
0.31 & 0.06 & 0.26 & -0.13 & 0 & -0.92 & 0.65 & -0.58\\
0.63 & 0.44 & 0.99 & 0.33 & -0.92 & 0 & 0.01 & -0.3 \\
-0.33 & 0.93 & 0 & -0.98 & 0.65 & 0.01 & 0 & 0.08\\
1 & -0.32 & -0.94 & 0.96 & -0.58 & -0.3 & 0.08 & 0
\end{pmatrix}
\begin{pmatrix}
1 \\ -1 \\ 1 \\ -1 \\ -1 \\ 1 \\ -1 \\ 0
\end{pmatrix}\\
=\frac12
\begin{pmatrix}
1 & -1 & 1 & -1 & -1 & 1 & -1 & 0
\end{pmatrix}
\begin{pmatrix}\frac{41}{20}\\ 
-\frac{87}{50}\\ 
\frac{119}{100}\\ 
\frac{57}{100}\\ 
-\frac{93}{100}\\ 
\frac{44}{25}\\ 
-\frac{23}{25}\\ 
-\frac{19}{50} \end{pmatrix}\\
=4.01
\end{align}
$$
### Calcul numérique

In [3]:
jastrow_parameters = [
0 , -0.7 , 0.28 , -0.42 , 0.31 , 0.63 , -0.33 , 1,
-0.7 , 0 , -0.11 , 0.38 , 0.06 , 0.44 , 0.93 , -0.32,
0.28 , -0.11 , 0 , -0.07 , 0.26 , 0.99 , 0 , -0.94,
-0.42 , 0.38 , -0.07 , 0 , -0.13 , 0.33 , -0.98 , 0.96,
0.31 , 0.06 , 0.26 , -0.13 , 0 , -0.92 , 0.65 , -0.58,
0.63 , 0.44 , 0.99 , 0.33 , -0.92 , 0 , 0.01 , -0.3,
-0.33 , 0.93 , 0 , -0.98 , 0.65 , 0.01 , 0 , 0.08,
1 , -0.32 , -0.94 , 0.96 , -0.58 , -0.3 , 0.08 , 0
]
jastrow_coef = imp.jastrow_exponent(spin_up, spin_down, jastrow_parameters, NSITES)
print(jastrow_coef)

jastrow_coef = imp.jastrow_fastupdate(
    jastrow_coef,
    jastrow_parameters,
    spin_up,
    spin_down,
    new_spin_up,
    spin_down,
    NSITES,
    1,
    0
)
print(jastrow_coef)

0.92
4.01


### Autres exemples avec différents nombre de sites
#### 2 Sites
$$
\begin{align}
\mathbf{v}&=
\begin{pmatrix}
0 & 0.27 \\ 0.27 & 0
\end{pmatrix}\\
n_\uparrow&=[01]\\
n_\downarrow&=[01]\\
\zeta&=[-1,1]\\
\frac12\zeta^T\mathbf{v}\zeta&=\frac12\begin{pmatrix}
-1 & 1
\end{pmatrix}\begin{pmatrix}
0.27 \\ -0.27
\end{pmatrix}=-0.27
\end{align}
$$
Il est important que les bits significatifs soient alignés le plus à gauche en mémoire. Il s'agit d'un détail d'implémentation étant donné l'utilisation de fonctions primitives telles que `leading_zeros()` pour augmenter la rapidité de l'adressage de bit individuels. Pour les calculs précédents, les $8$ bits du format `u8` étaient significatifs, d'où le fait que nous n'avions pas besoin de prendre ceci en compte.

In [4]:
params = [0 for i in range(64)]
params[1] = 0.27
params[2] = 0.27
nsites = 2
up = 1 << (8 - nsites)
down = 1 << (8 - nsites)
jastrow_coef = imp.jastrow_exponent(up, down, params, 2)
print(jastrow_coef)

-0.27


#### 3 Sites
$$
\begin{align}
\mathbf{v}&=
\begin{pmatrix}
0 & 1 & 1 \\ 1 & 0 & 1 \\ 1 & 1 & 0
\end{pmatrix}\\
n_\uparrow&=[010]\\
n_\downarrow&=[010]\\
\zeta&=[-1,1,-1]\\
\frac12\zeta^T\mathbf{v}\zeta&=\frac12\begin{pmatrix}
-1 & 1 & -1
\end{pmatrix}\begin{pmatrix}
0 \\ -2 \\ 0
\end{pmatrix}=-1
\end{align}
$$

In [5]:
params = [0 for i in range(64)]
params[1] = 1
params[2] = 1
params[3] = 1
params[5] = 1
params[6] = 1
params[7] = 1
nsites = 3
up = 2 << (8 - nsites)
down = 2 << (8 - nsites)
jastrow_coef = imp.compute_jastrow_easy_to_follow(up, down, params, nsites)
print(jastrow_coef)

-1.0


## Calcul du Produit Scalaire $\braket{x|\phi_{\text{Pfaff}}}$
L'état pfaffian est défini tel que
$$
\vert \phi_{\text{PF}}\rangle=
\left[
    \sum_{i, j = 0}^{N_s-1}F_{r_ir_j}^{\sigma_i\sigma_j}c_{i\sigma}^\dagger c_{j\sigma'}^\dagger
\right]^{N_e /2} \vert 0\rangle
$$
Ce qui nous intéresse est $\braket{x|\phi_{\text{PF}}}$. Cette valeur se calcule de la manière suivante:
$$
\begin{align}
\langle x\vert\phi_{\text{PF}}\rangle&=
\left(\frac{N_e}{2}\right)!\ \text{Pf}(X)\\
X_{ij}&=F_{r_ir_j}^{\sigma_i\sigma_j}-F_{r_jr_i}^{\sigma_j\sigma_i}
\end{align}
$$
#### Exemple à 3 sites
Pour se simplifier la vie, on choisi $n_{\uparrow}=[011]$ et $n_{\downarrow}=[101]$. On choisi aussi les paramètres variationnels

$$
\begin{align}
&F^{\uparrow\uparrow}=\begin{pmatrix}
0.4 & 0.7 & 0.8 \\
-1.0 & -0.3 & 0.4 \\
-0.2 & 0.8 & 0.9
\end{pmatrix}
&& F^{\uparrow\downarrow}=\begin{pmatrix}
0.4 & 0.1 & -0.1\\
-0.2 & 0.3 & -0.3\\
0.4 & -0.5 & 0.5
\end{pmatrix}\\
&F^{\downarrow\uparrow}=\begin{pmatrix}
0.2 & 0.3 &-1.0\\
0.8 & -0.4 & -0.5\\
-0.1 & 0.6 & 0.7
\end{pmatrix}
&& F^{\downarrow\downarrow}=\begin{pmatrix}
-1.0 & 0.8 & -0.9\\
0.7 & 0.4 & 0.5\\
-0.3 & -0.2 & 0.1
\end{pmatrix}
\end{align}
$$
La matrice $X$ est donc donnée par

$$
\begin{align}
X&=\begin{pmatrix}
0 & -0.4 & -0.5 & -0.9\\
0.4 & 0 & 1.4 & -0.2\\
0.5 & -1.4 & 0 & -0.6\\
0.9 & 0.2 & 0.6 & 0
\end{pmatrix}
\end{align}
$$
On calcule ensuite la pfaffian de cette matrice:

In [6]:
from pfapack import pfaffian as pf
import numpy as np
X = np.array([
    [0 , -0.4 , -0.5 , -0.9],
    [0.4 , 0 , 1.4 , -0.2],
    [0.5 , -1.4 , 0 , -0.6],
    [0.9 , 0.2 , 0.6 , 0]
])
pf.pfaffian(X)

-1.12

Ce qui signifie que nous avons

$$
\begin{align}
\braket{x|\phi_{\text{PF}}}&=\left(\frac{4}{2}\right)!\cdot-1.12\\
&=-2.24
\end{align}
$$

On peut ensuite faire le hopping pour obtenir $n_{\uparrow}=[101]$, ce qui nous donne maintenant la matrice

$$
\begin{align}
X&=\begin{pmatrix}
0 & 1.0 & -0.2 & 0\\
-1.0 & 0 & 1.4 & -0.2\\
0.2 & -1.4 & 0 & -0.6\\
0 & 0.2 & 0.6 & 0
\end{pmatrix}
\end{align}
$$

In [7]:
X = np.array([
    [0 , 1.0 , 0.2 , 0],
    [-1.0 , 0 , 1.4 , -0.2],
    [-0.2 , -1.4 , 0 , -0.6],
    [0 , 0.2 , 0.6 , 0]
])
pf.pfaffian(X)

-0.5599999999999999

Ce qui signifie que nous avons

$$
\begin{align}
\braket{x|\phi_{\text{PF}}}&=\left(\frac{4}{2}\right)!\cdot-0.56\\
&=-1.12
\end{align}
$$

In [8]:
fij = [
    #0.4 , 0.7 , 0.8 ,
    #-1.0 , -0.3 , 0.4 ,
    #-0.2 , 0.8 , 0.9 ,
    0,0,0,
    0,0,0,
    0,0,0,
0.4 , 0.1 , -0.1,
    -0.2 , 0.3 , -0.3,
    0.4 , -0.5 , 0.5 ,
    0.2 , 0.3 ,-1.0,
    0.8 , -0.4 , -0.5,
    -0.1 , 0.6 , 0.7 ,
    0,0,0,
    0,0,0,
    0,0,0,
    #-1.0 , 0.8 , -0.9,
    #0.7 , 0.4 , 0.5,
    #-0.3 , -0.2 , 0.1
]
nsites = 3
up = 3 << (8 - nsites)
down = 5 << (8 - nsites)
scal = imp.compute_internal_product_py(up, down, fij, nsites)
print(scal)

up = 5 << (8 - nsites)
scal = imp.compute_internal_product_py(up, down, fij, nsites)
print(scal)

Fij: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.1, -0.1, -0.2, 0.3, -0.3, 0.4, -0.5, 0.5, 0.2, 0.3, -1.0, 0.8, -0.4, -0.5, -0.1, 0.6, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
X: (0.1 - 0.8) at (2, 0), (-0.1 - -0.1) at (2, 1), 
(-0.5 - -0.5) at (3, 0), (0.5 - 0.7) at (3, 1), 


thread '<unnamed>' panicked at 'explicit panic', src/pfaffian.rs:174:5
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: explicit panic

## Calcul de la Densité $\rho(x)$